# After years of thought it's finally here

# This is the code that will run at runtime; every elite run of the challenger

The preprocessing code only comes into effect when mons are added to or removed from the database entirely

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# This dictionary maps attacking types to their coverage
with open(r"jsonfiles/Off.json","r") as p:
    Off=json.load(p)
print(Off)

{'Typeless': [], 'Normal': [], 'Fire': ['Grass', 'Ice', 'Bug', 'Steel'], 'Water': ['Fire', 'Ground', 'Rock'], 'Grass': ['Water', 'Ground', 'Rock'], 'Electric': ['Water', 'Flying'], 'Ice': ['Grass', 'Ground', 'Flying', 'Dragon'], 'Fighting': ['Normal', 'Ice', 'Rock', 'Dark', 'Steel'], 'Poison': ['Grass', 'Fairy'], 'Ground': ['Fire', 'Electric', 'Poison', 'Rock', 'Steel', 'Dragon'], 'Flying': ['Grass', 'Fighting', 'Bug'], 'Psychic': ['Fighting', 'Poison'], 'Bug': ['Grass', 'Psychic', 'Dark'], 'Rock': ['Fire', 'Ice', 'Flying', 'Bug', 'Dragon'], 'Ghost': ['Psychic', 'Ghost'], 'Dragon': ['Dragon'], 'Dark': ['Psychic', 'Ghost'], 'Steel': ['Ice', 'Rock', 'Fairy'], 'Fairy': ['Fighting', 'Dragon', 'Dark'], 'nan': []}


In [3]:
# This dictionary maps move names to their types

with open(r"jsonfiles/move_dict.json","r") as p:
    move_dict=json.load(p)
print(move_dict)

{'Acrobatics': 'Flying', 'Aerial Ace': 'Flying', 'Air Slash': 'Flying', 'Alluring Voice': 'Fairy', 'Aqua Cutter': 'Water', 'Aqua Jet': 'Water', 'Aqua Tail': 'Water', 'Assurance': 'Dark', 'Aura Sphere': 'Fighting', 'Aurora Beam': 'Ice', 'Avalanche': 'Ice', 'Axe Kick': 'Fighting', 'Beat Up': 'Dark', 'Belch': 'Poison', 'Bite': 'Dark', 'Blaze Kick': 'Fire', 'Body Press': 'Fighting', 'Bone Club': 'Ground', 'Bone Rush': 'Ground', 'Bonemerang': 'Ground', 'Boomburst': 'Normal', 'Bounce': 'Flying', 'Brave Bird': 'Flying', 'Breaking Swipe': 'Dragon', 'Brick Break': 'Fighting', 'Brine': 'Water', 'Brutal Swing': 'Dark', 'Bubble Beam': 'Water', 'Bug Buzz': 'Bug', 'Bulldoze': 'Ground', 'Bullet Punch': 'Steel', 'Bullet Seed': 'Grass', 'Burning Jealousy': 'Fire', 'Chilling Water': 'Water', 'Close Combat': 'Fighting', 'Comeuppance': 'Typeless', 'Counter': 'Typeless', 'Crabhammer': 'Water', 'Cross Chop': 'Fighting', 'Cross Poison': 'Poison', 'Crunch': 'Dark', 'Dark Pulse': 'Dark', 'Darkest Lairat': 'Dar

In [4]:
# This dataframe has everything you'd need

core_df = pd.read_json(r"jsonfiles/core_df.json")
core_df.head()

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,None,RU,None,None,None,Iron Tail,Rock Slide,Earthquake,...,None,None,None,None,None,None,None,None,None,None
Absol,Dark,None,None,RU,None,None,None,Counter,Iron Tail,Rock Slide,...,None,None,None,None,None,None,None,None,None,None
Accelgor,Bug,None,None,RU,None,None,None,Water Shuriken,Sludge Bomb,Venoshock,...,None,None,None,None,None,None,None,None,None,None
Aerodactyl,Rock,Flying,None,RU,None,None,None,Aqua Tail,Iron Head,Psychic Fangs,...,None,None,None,None,None,None,None,None,None,None
Aggron,Steel,Rock,None,RU,None,Yes,None,Aqua Tail,Surf,Counter,...,None,None,None,None,None,None,None,None,None,None


In [5]:
#Type Offense dictionary
#mirror coat and counter are typeless
Off={
    'Typeless':[],
    'Normal':[],
    'Fire':['Grass','Ice','Bug','Steel'],
    'Water':['Fire','Ground','Rock'],
    'Grass':['Water','Ground','Rock'],
    'Electric':['Water','Flying'],
    'Ice':['Grass','Ground','Flying','Dragon'],
    'Fighting':['Normal','Ice','Rock','Dark','Steel'],
    'Poison':['Grass','Fairy'],
    'Ground':['Fire','Electric','Poison','Rock','Steel','Dragon'],    #ground is good against nearly half dragons so I added that
    'Flying':['Grass','Fighting','Bug'],
    'Psychic':['Fighting','Poison'],
    'Bug':['Grass','Psychic','Dark'],
    'Rock':['Fire','Ice','Flying','Bug','Dragon'],   #rock is good against nearly half dragons so I added that
    'Ghost':['Psychic','Ghost'],
    'Dragon':['Dragon'],
    'Dark':['Psychic','Ghost'],
    'Steel':['Ice','Rock','Fairy'],
    'Fairy':['Fighting','Dragon','Dark'],
    'nan' :[]
}
#notes: freeze dry is sneakily coded as an electric type move to include super effectiveness on water

In [6]:
core_df

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,None,RU,None,None,None,Iron Tail,Rock Slide,Earthquake,...,None,None,None,None,None,None,None,None,None,None
Absol,Dark,None,None,RU,None,None,None,Counter,Iron Tail,Rock Slide,...,None,None,None,None,None,None,None,None,None,None
Accelgor,Bug,None,None,RU,None,None,None,Water Shuriken,Sludge Bomb,Venoshock,...,None,None,None,None,None,None,None,None,None,None
Aerodactyl,Rock,Flying,None,RU,None,None,None,Aqua Tail,Iron Head,Psychic Fangs,...,None,None,None,None,None,None,None,None,None,None
Aggron,Steel,Rock,None,RU,None,Yes,None,Aqua Tail,Surf,Counter,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,None,RU,None,None,None,Psychic,Psychic Noise,Solar Beam,...,None,None,None,None,None,None,None,None,None,None
Zangoose,Normal,None,None,RU,None,None,None,Surf,Counter,Rock Slide,...,Payback,Pursuit,Power Trip,X Scissor,None,None,None,None,None,None
Zarude,Dark,Grass,None,RU,None,None,None,Iron Tail,Rock Slide,Rock Tomb,...,None,None,None,None,None,None,None,None,None,None
Zebstrika,Electric,None,None,RU,None,None,None,Smart Strike,High Horsepower,Bulldoze,...,None,None,None,None,None,None,None,None,None,None


In [7]:
# This code has been copied from the other pokemon file

pre_coverage=core_df.drop(columns=['Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

coverage=pre_coverage.map(lambda x: Off.get(x))

elite_1='Ground'
elite_2='Psychic'
elite_3='Fairy'
elite_4='Fire'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))
final =core_df[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]
final

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,NaN,NaN,NaN,NaN,NaN,Iron Tail,Rock Slide,Earthquake,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,NaN,NaN,NaN,NaN,Water Shuriken,Sludge Bomb,Venoshock,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,NaN,NaN,NaN,NaN,NaN,NaN,Aqua Tail,Iron Head,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,NaN,NaN,NaN,NaN,Aqua Tail,Surf,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Altaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Steel Wing,Ice Beam,Earthquake,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wishiwashi,Water,NaN,NaN,NaN,NaN,NaN,NaN,Iron Tail,Ice Beam,Earthquake,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wyrdeer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Iron Tail,Earthquake,High Horsepower,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Surf,NaN,Rock Slide,...,Payback,Pursuit,Power Trip,X Scissor,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,NaN,NaN,NaN,NaN,Iron Tail,Rock Slide,Rock Tomb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


create a filter later so that moves that rely on the significantly weaker stat can be removed if desired

so then that means we create a column for a pokemon, which states its dominant offense, and if no dominant offense then it's empty

my latest idea on this is to add the physical and special attack of every mon, and sort based on this.

remember to somehow let counter and mirror coat and metal burst maybe remain unfiltered as strong moves

maybe we should remember when even special attackers have moves like uturn, or physical attackers have moves like volt switch Both of these should count as supereffective good moves because the BP exceeds 60.

I should actually create a whole ass column called pivot which shows when a mon has a pivot move or not, and what this/these priority move is/are This requires me to type in uturn for a bunch of bugs too. (and volt switch/flip turn). Also need to code in parting shot, a non offensive move.

and priority moves also should be available to both, cuz the point is usually just to break sash you know what, maybe I should make a column called priority and add in all the normal priority moves like quick attack too. I probably won't add grassy glide, but I'll force add Comfey's draining moves as priority particulary for it.

do all these complicated actions with df.query

one filter should also make sure that a type is stab supereffective for a mon (but also remember that freeze dry is electric type and needs to be counted as stab)

vectorized method usually means you create another column, like df['result']=df[first] + df[second]

Also don't heavily deprioritize solar beam and stuff, cuz z moves are a thing. Maybe mention it in your application that the recommended strategy is to z move

have a filter to avoid inconsistent moves like fling, pursuit
maybe bulldoze and rock tomb too, although those moves could come under another filter

please try to sort the output and group it by the type, so that all dark moves are together, all fighting moves are together, etc.
(maybe you can also have a priority list for the moves of each type, and then display the moves in the order of the priority list)

copy paste technician mons before and after all the move deletions so that they don't lose their moves
Also Strong Jaw mons and stuff ig

In [8]:
clean_df=clean_df.replace(to_replace=["Fling","Pursuit","Bulldoze","Rock Tomb","Dig"],value=np.NaN)

# this was the cell I used to make certain cells null

NameError: name 'clean_df' is not defined